# Stage I GAN

In [0]:
## this is file of stage 1 of stackgan.
## pre-requirements : download birds.zip file
## resumable training is added to the code
## go-to cell number 5 and change variable as you need. 

# Importing Libraries

In [0]:
###### javascript function
## connect-push ---- paste this in consle

function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
###### birds dataset for dis.

#### download birds.zip form given link
## mega link ----  https://mega.nz/file/T4dlRKoL#GrqNQm-c2F8iGyQ5HfvlbzIhI7yJJPFnPdDWyavlAv8

!cp /content/drive/'My Drive'/GAN/birds.zip /content/    ### change first arg to birds.zip file in drive
!unzip birds.zip

In [0]:
#### change variable as per traning of model

previousmodel = '3005'  ### previous model (set as date)
currentmodel = '1106'   ### current model (set as date)
modelpath = 'date3005/'   ### drive directory to fetch previous trained model
weightpath = 'date1106/'   ### create directory in drive to store models
createdir = 'date1106/models/'   ### store models
dirpath = '/content/drive/"My Drive"/GAN/'   ### drive path to store current models and its weights
endpath = 'models/*'   ### for copy command


SET_EPOCHS = 2   ### number of epochs (set default to 300)

In [0]:
## load model from drive
!mkdir -p /content/models/
###### copy model 
!cp -r $dirpath$modelpath$endpath /content/models/

In [0]:
import os
import pickle
import random
import time

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from keras import Input, Model
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, \
    concatenate, Flatten, Lambda, Concatenate
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from keras.models import load_model
from keras.models import model_from_json

# Loading of Dataset

In [0]:
def load_class_ids(class_info_file_path):
    """
    Load class ids from class_info.pickle file
    """
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encoding='latin1')
        return class_ids

In [0]:
def load_embeddings(embeddings_file_path):
    """
    Load embeddings
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings

In [0]:
def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

In [0]:
def load_bounding_boxes(dataset_dir):
    """
    Load bounding boxes and return a dictionary of file names and corresponding bounding boxes
    """
    # Paths
    bounding_boxes_path = os.path.join(dataset_dir, 'bounding_boxes.txt')
    file_paths_path = os.path.join(dataset_dir, 'images.txt')

    # Read bounding_boxes.txt and images.txt file
    df_bounding_boxes = pd.read_csv(bounding_boxes_path,
                                    delim_whitespace=True, header=None).astype(int)
    df_file_names = pd.read_csv(file_paths_path, delim_whitespace=True, header=None)

    # Create a list of file names
    file_names = df_file_names[1].tolist()

    # Create a dictionary of file_names and bounding boxes
    filename_boundingbox_dict = {img_file[:-4]: [] for img_file in file_names[:2]}

    # Assign a bounding box to the corresponding image
    for i in range(0, len(file_names)):
        # Get the bounding box
        bounding_box = df_bounding_boxes.iloc[i][1:].tolist()
        key = file_names[i][:-4]
        filename_boundingbox_dict[key] = bounding_box

    return filename_boundingbox_dict


In [0]:
def get_img(img_path, bbox, image_size):
    """
    Load and resize image
    """
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - R)
        y2 = np.minimum(height, center_y + R)
        x1 = np.maximum(0, center_x - R)
        x2 = np.minimum(width, center_x + R)
        img = img.crop([x1, y1, x2, y2])
    img = img.resize(image_size, PIL.Image.BILINEAR)
    return img

In [0]:
def load_dataset(filenames_file_path, class_info_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    """
    Load dataset
    """
    filenames = load_filenames(filenames_file_path)
    class_ids = load_class_ids(class_info_file_path)
    bounding_boxes = load_bounding_boxes(cub_dataset_dir)
    all_embeddings = load_embeddings(embeddings_file_path)

    X, y, embeddings = [], [], []

    print("Embeddings shape:", all_embeddings.shape)

    for index, filename in enumerate(filenames):
        bounding_box = bounding_boxes[filename]

        try:
            # Load images
            img_name = '{}/images/{}.jpg'.format(cub_dataset_dir, filename)
            img = get_img(img_name, bounding_box, image_size)

            all_embeddings1 = all_embeddings[index, :, :]

            embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
            embedding = all_embeddings1[embedding_ix, :]

            X.append(np.array(img))
            y.append(class_ids[index])
            embeddings.append(embedding)
        except Exception as e:
            print(e)

    X = np.array(X)
    y = np.array(y)
    embeddings = np.array(embeddings)
    return X, y, embeddings

# Model Creation

In [0]:
def generate_c(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape=K.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c

In [0]:
def build_ca_model():
    """
    Get conditioning augmentation model.
    Takes an embedding of shape (1024,) and returns a tensor of shape (256,)
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [0]:
def build_embedding_compressor_model():
    """
    Build embedding compressor model
    """
    input_layer = Input(shape=(1024,))
    x = Dense(128)(input_layer)
    x = ReLU()(x)

    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [0]:
def build_stage1_generator():
    """
    Builds a generator model used in Stage-I
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)

    c = Lambda(generate_c)(mean_logsigma)

    input_layer2 = Input(shape=(100,))

    gen_input = Concatenate(axis=1)([c, input_layer2])

    x = Dense(128 * 8 * 4 * 4, use_bias=False)(gen_input)
    x = ReLU()(x)

    x = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation(activation='tanh')(x)

    stage1_gen = Model(inputs=[input_layer, input_layer2], outputs=[x, mean_logsigma])
    return stage1_gen


In [0]:
def build_stage1_discriminator():
    """
    Create a model which takes two inputs
    1. One from above network
    2. One from the embedding layer
    3. Concatenate along the axis dimension and feed it to the last module which produces final logits
    """
    input_layer = Input(shape=(64, 64, 3))

    x = Conv2D(64, (4, 4),
               padding='same', strides=2,
               input_shape=(64, 64, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    input_layer2 = Input(shape=(4, 4, 128))

    merged_input = concatenate([x, input_layer2])

    x2 = Conv2D(64 * 8, kernel_size=1,
                padding="same", strides=1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)

    stage1_dis = Model(inputs=[input_layer, input_layer2], outputs=[x2])
    return stage1_dis

In [0]:
def build_adversarial_model(gen_model, dis_model):
    input_layer = Input(shape=(1024,))
    input_layer2 = Input(shape=(100,))
    input_layer3 = Input(shape=(4, 4, 128))

    x, mean_logsigma = gen_model([input_layer, input_layer2])

    dis_model.trainable = False
    valid = dis_model([x, input_layer3])

    model = Model(inputs=[input_layer, input_layer2, input_layer3], outputs=[valid, mean_logsigma])
    return model

# Defining Loss

In [0]:
def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss

In [0]:

def custom_generator_loss(y_true, y_pred):
    # Calculate binary cross entropy loss
    return K.binary_crossentropy(y_true, y_pred)


In [0]:
def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

In [0]:
def write_log(callback, name, loss, batch_no):
    """
    Write training summary to TensorBoard
    """
    summary = tf.Summary()
    summary_value = summary.value.add()
    summary_value.simple_value = loss
    summary_value.tag = name
    callback.writer.add_summary(summary, batch_no)
    callback.writer.flush()

# Downloading of dataset in colab

In [0]:
!wget http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz

In [0]:
!ls

In [0]:
import tarfile
tar = tarfile.open("CUB_200_2011.tgz")
tar.extractall()
tar.close()

In [0]:
!ls

In [0]:
cd CUB_200_2011

In [0]:
!ls

In [0]:
os.chdir("/content/")

In [0]:
!ls

# **My Edit__dir_create**

In [0]:
result_path = '/content/results'+currentmodel+'/'
!mkdir -p /content/logs/
!mkdir -p $result_path

In [0]:
!mkdir -p $dirpath$createdir

# Main File

In [0]:
if __name__ == '__main__':
    gen_loss = open('/content/gen_loss_'+currentmodel+'.txt','a')
    dis_loss = open('/content/dis_loss_'+currentmodel+'.txt','a')
    data_dir = "/content/birds/"
    train_dir = data_dir + "/train"
    test_dir = data_dir + "/test"
    image_size = 64
    batch_size = 64
    z_dim = 100
    stage1_generator_lr = 0.0002
    stage1_discriminator_lr = 0.0002
    stage1_lr_decay_step = 600
    epochs = SET_EPOCHS
    condition_dim = 128

    embeddings_file_path_train = train_dir + "/char-CNN-RNN-embeddings.pickle"
    embeddings_file_path_test = test_dir + "/char-CNN-RNN-embeddings.pickle"

    filenames_file_path_train = train_dir + "/filenames.pickle"
    filenames_file_path_test = test_dir + "/filenames.pickle"

    class_info_file_path_train = train_dir + "/class_info.pickle"
    class_info_file_path_test = test_dir + "/class_info.pickle"

    cub_dataset_dir = "/content/CUB_200_2011"
    
    # Define optimizers
    dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
    gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

    """"
    Load datasets
    """
    X_train, y_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                      class_info_file_path=class_info_file_path_train,
                                                      cub_dataset_dir=cub_dataset_dir,
                                                      embeddings_file_path=embeddings_file_path_train,
                                                      image_size=(64, 64))

    X_test, y_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                   class_info_file_path=class_info_file_path_test,
                                                   cub_dataset_dir=cub_dataset_dir,
                                                   embeddings_file_path=embeddings_file_path_test,
                                                   image_size=(64, 64))

    """
    Build and compile networks
    """
    print("===========build models==================")
    try:
      ca_model = load_model('/content/models/ca_model_'+previousmodel+'.h5')
      print('load ca_model')
    except:
      ca_model = build_ca_model()
      ca_model.compile(loss="binary_crossentropy", optimizer="adam")
      print('create ca_model')

    try:
      stage1_dis = load_model('/content/models/stage1_dis_model_'+previousmodel+'.h5')
      print('load stage1_dis')
    except:
      stage1_dis = build_stage1_discriminator()
      stage1_dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer)
      print('create stage1_dis')

    try:
      stage1_gen = load_model('/content/models/stage1_gen_model_'+previousmodel+'.h5')
      print('load stage1_gen')
    except:
      stage1_gen = build_stage1_generator()
      stage1_gen.compile(loss="mse", optimizer=gen_optimizer)
      print('create stage1_gen')

    try:
      embedding_compressor_model = load_model('/content/models/embedding_compressor_model_'+previousmodel+'.h5')
      print('load embedding_compressor_model')
    except:
      embedding_compressor_model = build_embedding_compressor_model()
      embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")
      print('create embedding_compressor_model')

    try:
      # load json and create model
      json_file = open('/content/models/adversarial_model_json_'+previousmodel+'.json', 'r')
      loaded_model_json = json_file.read()
      json_file.close()
      adversarial_model = model_from_json(loaded_model_json)
      # load weights into new model
      adversarial_model.load_weights('/content/models/adversarial_model_weight_'+previousmodel+'.h5')
      adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],optimizer=gen_optimizer, metrics=None)
      print('load adversarial_model')
    except:
      adversarial_model = build_adversarial_model(gen_model=stage1_gen, dis_model=stage1_dis)
      adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],optimizer=gen_optimizer, metrics=None)
      print('create adversarial_model')

    tensorboard = TensorBoard(log_dir="logs/".format(time.time()))
    tensorboard.set_model(stage1_gen)
    tensorboard.set_model(stage1_dis)
    tensorboard.set_model(ca_model)
    tensorboard.set_model(embedding_compressor_model)

    


    # Generate an array containing real and fake values
    # Apply label smoothing as well
    real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
    fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1

    
    for epoch in range(epochs):
        print("========================================")
        print("Epoch is:", epoch)
        print("Number of batches", int(X_train.shape[0] / batch_size))

        gen_losses = []
        dis_losses = []

        # Load data and train model
        number_of_batches = int(X_train.shape[0] / batch_size)
        for index in range(number_of_batches):
            print("Batch:{}".format(index+1))
            
            """
            Train the discriminator network
            """
            # Sample a batch of data
            z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
            image_batch = X_train[index * batch_size:(index + 1) * batch_size]
            embedding_batch = embeddings_train[index * batch_size:(index + 1) * batch_size]
            image_batch = (image_batch - 127.5) / 127.5

            # Generate fake images
            fake_images, _ = stage1_gen.predict([embedding_batch, z_noise], verbose=3)

            # Generate compressed embeddings
            compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
            compressed_embedding = np.reshape(compressed_embedding, (-1, 1, 1, condition_dim))
            compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))

            dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],
                                                      np.reshape(real_labels, (batch_size, 1)))
            dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],
                                                      np.reshape(fake_labels, (batch_size, 1)))
            dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],
                                                       np.reshape(fake_labels[1:], (batch_size-1, 1)))

            d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))

            #print("d_loss_real:{}".format(dis_loss_real))
            #print("d_loss_fake:{}".format(dis_loss_fake))
            #print("d_loss_wrong:{}".format(dis_loss_wrong))
            #print("d_loss:{}".format(d_loss))

            """
            Train the generator network 
            """
            g_loss = adversarial_model.train_on_batch([embedding_batch, z_noise, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])
            print("g_loss:{}".format(g_loss))

            dis_losses.append(d_loss)
            gen_losses.append(g_loss)

        """
        Save losses to Tensorboard after each epoch
        """
        #write_log(tensorboard, 'discriminator_loss', np.mean(dis_losses), epoch)
        #write_log(tensorboard, 'generator_loss', np.mean(gen_losses[0]), epoch)
        
        dis_loss.write(str(np.mean(dis_losses))+'\n')
        gen_loss.write(str(np.mean(gen_losses[0]))+'\n')

        # Generate and save images after every 2nd epoch
        if epoch % 2 == 0:
            # z_noise2 = np.random.uniform(-1, 1, size=(batch_size, z_dim))
            z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
            embedding_batch = embeddings_test[0:batch_size]
            fake_images, _ = stage1_gen.predict_on_batch([embedding_batch, z_noise2])

            # Save images
            for i, img in enumerate(fake_images[:10]):
                save_rgb_img(img, "results"+currentmodel+"/gen_{}_{}.png".format(epoch, i))
    
    ### close file
    dis_loss.close()
    gen_loss.close()
    
    '''
    # Save weights
    stage1_gen.save_weights("stage1_gen_1905.h5")
    stage1_dis.save_weights("stage1_dis_1905.h5")
    #!cp -r /content/stage1_gen.h5 /content/drive/'My Drive'/GAN/date1105/
    #!cp -r /content/stage1_dis.h5 /content/drive/'My Drive'/GAN/date1105/
    #!cp -r /content/logs /content/drive/'My Drive'/GAN/date1105/
    #!cp -r /content/results /content/drive/'My Drive'/GAN/date1105/

    # Save MOdels
    stage1_gen.save("stage1_gen_model_1905.h5")
    stage1_dis.save("stage1_dis_model_1905.h5")
    ca_model.save("ca_model_1905.h5")
    embedding_compressor_model.save("embedding_compressor_model_1905.h5")    

    adversarial_model.save_weights("adversarial_model_weight_1905.h5")
    adversarial_model_json = adversarial_model.to_json()
    with open("adversarial_model_json_1905.json", "w") as json_file:
      json_file.write(adversarial_model_json)
    '''


In [0]:
# make directory to save current model and weights
!mkdir -p /content/currentweights/
!mkdir -p /content/currentmodels/

# Save weights
stage1_gen.save_weights("currentweights/stage1_gen_"+currentmodel+".h5")
stage1_dis.save_weights("currentweights/stage1_dis_"+currentmodel+".h5")


# Save MOdels
stage1_gen.save("currentmodels/stage1_gen_model_"+currentmodel+".h5")
stage1_dis.save("currentmodels/stage1_dis_model_"+currentmodel+".h5")
ca_model.save("currentmodels/ca_model_"+currentmodel+".h5")
embedding_compressor_model.save("currentmodels/embedding_compressor_model_"+currentmodel+".h5")    

adversarial_model.save_weights("currentmodels/adversarial_model_weight_"+currentmodel+".h5")
adversarial_model_json = adversarial_model.to_json()
with open("currentmodels/adversarial_model_json_"+currentmodel+".json", "w") as json_file:
  json_file.write(adversarial_model_json)

In [0]:
gen_los = '/content/gen_loss_'+currentmodel+'.txt'
dis_los = '/content/dis_loss_'+currentmodel+'.txt'

In [0]:
######### copy weights
!cp -r /content/currentweights/* $dirpath$weightpath

######## copy loss
!cp -r $gen_los $dirpath$weightpath
!cp -r $dis_los $dirpath$weightpath

######## copy results
!cp -r /content/logs /content/drive/'My Drive'/GAN/$weightpath
!cp -r $result_path /content/drive/'My Drive'/GAN/$weightpath

###### copy model 
!cp -r /content/currentmodels/* $dirpath$createdir